In [3]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357273 sha256=79a05a98a28ef44f6d6836287f89977978155cc91a786772742f6db8234c28cc
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [10]:
# 1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise.
from surprise import Dataset

# Завантаження датасету ML-100K
data = Dataset.load_builtin('ml-100k')


In [11]:
# 2. Виведіть перші 5 рядків завантаженого датасету.
import pandas as pd

# Отримання даних у форматі DataFrame
ratings = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])
print(ratings.head())


  user_id item_id  rating  timestamp
0     196     242     3.0  881250949
1     186     302     3.0  891717742
2      22     377     1.0  878887116
3     244      51     2.0  880606923
4     166     346     1.0  886397596


In [12]:
# 3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.
from surprise import KNNBasic, SVD
from surprise.model_selection import train_test_split

# Розділення даних на тренувальний і тестовий набори
trainset, testset = train_test_split(data, test_size=0.2)

# Визначення алгоритмів
algo_knn = KNNBasic()
algo_svd = SVD()

# Навчання алгоритмів
algo_knn.fit(trainset)
algo_svd.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [13]:
# 4. Використайте крос-валідацію для підбору оптимальних параметрів для обох алгоритмів.
from surprise.model_selection import cross_validate

# Крос-валідація для KNN
results_knn = cross_validate(algo_knn, data, measures=['MAE'], cv=5, verbose=True)

# Крос-валідація для SVD
results_svd = cross_validate(algo_svd, data, measures=['MAE'], cv=5, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7705  0.7686  0.7714  0.7768  0.7764  0.7728  0.0033  
Fit time          0.31    0.34    0.35    0.34    0.36    0.34    0.02    
Test time         4.19    3.89    3.18    4.30    3.27    3.77    0.46    
Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7421  0.7334  0.7343  0.7370  0.7446  0.7383  0.0044  
Fit time          1.39    1.44    1.42    1.97    1.70    1.58    0.23    
Test time

In [14]:
mean_mae_knn = results_knn['test_mae'].mean()
mean_mae_svd = results_svd['test_mae'].mean()

print(f"Середня MAE для KNN: {mean_mae_knn}")
print(f"Середня MAE для SVD: {mean_mae_svd}")

best_algo = "KNN" if mean_mae_knn < mean_mae_svd else "SVD"
print(f"Найкращий алгоритм: {best_algo}")


Середня MAE для KNN: 0.7727518696169297
Середня MAE для SVD: 0.7382743831365934
Найкращий алгоритм: SVD


In [24]:
# Вибір алгоритму для рекомендацій
selected_algo = algo_knn if best_algo == "KNN" else algo_svd

# Отримання рекомендацій
user_id = '196'
item_ids = trainset.all_items()
recommendations = [(item_id, selected_algo.predict(user_id, item_id).est) for item_id in item_ids]
recommendations.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих рекомендацій
top_10 = recommendations[:10]
print("Рекомендовані фільми:", top_10)


Рекомендовані фільми: [(0, 3.538414330855143), (1, 3.538414330855143), (2, 3.538414330855143), (3, 3.538414330855143), (4, 3.538414330855143), (5, 3.538414330855143), (6, 3.538414330855143), (7, 3.538414330855143), (8, 3.538414330855143), (9, 3.538414330855143)]


In [16]:
from surprise import Reader

# Читач даних
reader = Reader(rating_scale=(1, 5))
data_custom = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)

# Створення та навчання моделі
trainset_custom = data_custom.build_full_trainset()
algo_custom = SVD()
algo_custom.fit(trainset_custom)


In [23]:
# 6. Виведи рекомендації (10 фільмів) для конкретного користувача.
# Оцінка моделі
testset_custom = trainset_custom.build_anti_testset()
predictions = algo_custom.test(testset_custom)

# Отримання рекомендації для користувача
user_id_custom = '196'
recommendations_custom = [(pred.iid, pred.est) for pred in predictions if pred.uid == user_id_custom]
recommendations_custom.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих рекомендацій
top_10_custom = recommendations_custom[:10]
print("Рекомендації для користувача:", top_10_custom)


Рекомендації для користувача: [('169', 4.688654296561971), ('483', 4.602161847912318), ('408', 4.521920388510804), ('923', 4.443235805313929), ('272', 4.424896565747315), ('100', 4.421809182148573), ('98', 4.402302791971666), ('657', 4.372525535201296), ('963', 4.372409425462389), ('189', 4.361459828479179)]


Висновок: Лабораторна робота продемонструвала важливість рекомендаційних систем у сучасному світі та їхню здатність адаптуватися до індивідуальних уподобань користувачів. Вивчені методи та алгоритми можуть бути використані для подальших досліджень і практичного застосування в різних галузях, таких як електронна комерція, медіа та соціальні мережі.